In [ ]:
#%%
import numpy as np
import theano
import theano.tensor as T

# Introduction to Theano

Theano is a library for doing symbolic computaion. To understand this a comparison
with numpy as probably best...

Lets create to matrices and calculate their product..

In [ ]:
#%%
x = np.random.rand(2,3)
y = np.random.rand(3,5)

z = np.dot(x,y)
print(z)

So what happened there? We made two matrices and in the process space was allocated
and filled with some random numbers. We then calculated their product, some more
space was allocated for the resulut and then the actual computaion took place!

Lets do the same but using Theano...

In [ ]:
#%%
x = T.matrix()
y = T.matrix()

z = T.dot(x,y)
print(z)

It looks similar but is in fact very different. No space was allocated no numbers
exist within the matrices, no computation took place. The two variables x,y are just
boxes which at some point in the future can contain data - the size of the matrices has
not yet been specified. The result z will take the contents of the boxes and at some
later point calculate the result, usually on a GPU. Theano is lazy...

How to actualy do the computation? We use a theano function...

In [ ]:
#%%
f = theano.function([x,y],z)

The first argument is a list of input variables - the boxes we created. The second
argument is the output, in this case z. To use this function we need to specify some
data...

In [ ]:
#%%
x_ = np.random.rand(2,3).astype('float32')
y_ = np.random.rand(3,6).astype('float32')

Its really important that we convert to 32 bit float since Theano is used on GPU's
and most do not have support for 64 bit arithmetic...

We can now use this function...we should get a matrix of size 2x6

In [ ]:
#%%
print(f(x_,y_))

This function will work with matrices of any size as long as the dimensions
agree (the last dim of x should be the same as first dim of y)

In [ ]:
#%%
x_ = np.random.rand(200,300).astype('float32')
y_ = np.random.rand(300,600).astype('float32')

print(f(x_,y_))

In [ ]:
#%%
x_ = np.random.rand(200,30).astype('float32')
y_ = np.random.rand(300,600).astype('float32')

print(f(x_,y_))

We get an error due to dimension mismatch...

## Scalars, Vectors and Matrices
Theano can manipulate various data types. Lets do some more complex things...

In [ ]:
#%%
a = T.scalar()
a_ = 2
b = T.scalar()
b_ = 56

u = T.vector()
u_ = np.random.rand(5).astype('float32')
v = T.vector()
v_ = np.ones(5).astype('float32')

A = T.matrix()
A_= np.eye(5).astype('float32')
B = T.matrix()
B_ = np.random.rand(5,5).astype('float32')

f1 = a + b
f2 = T.sin(a)

f3 = T.dot(u,v)
f4 = a*v

f5 = A + B
f6 = b*A
f7 = T.dot(B,u)

f1_ = theano.function([a,b],f1)
f2_ = theano.function([a],f2)
f3_ = theano.function([u,v],f3)
f4_ = theano.function([a,v],f4)
f5_ = theano.function([A,B],f5)
f6_ = theano.function([b,A],f6)
f7_ = theano.function([B,u],f7)

print(f1_(a_,b_))
print(f2_(a_))
print(f3_(u_,v_))
print(f4_(a_,v_))
print(f5_(A_,B_))
print(f6_(b_,A_))
print(f7_(B_,u_))

We can form large chains of computation with various intermediate steps...

In [ ]:
#%%
a = T.scalar()
A = T.matrix()
B = T.matrix()

Z = a*A - a*B
Y = T.dot(A,-B)

W = Z + Y

res = theano.function([a,A,B],W)

a_ = 10
A_ = np.random.rand(5,5).astype('float32')
B_ = np.random.rand(5,5).astype('float32')

print(res(a_,A_,B_))

## Derivatives
Theano does derivatives for us, no need for algebra!

Lets find the derivative of the following;
$$ y = sin(x) $$
$$ \frac{dy}{dx} = cos(x) $$